In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import tensorflow as tf

# Add the project root to the path so we can import 'src'
sys.path.append(os.path.abspath('..'))

from src.config import Config
from src.data.dataset import SubwayDataGenerator
from src.models.st_covnet import HeadwayConvLSTM
from src.training.trainer import Trainer
from src.evaluator import Evaluator

print(f"Tensorflow Version:{tf.__version__}")
print(f"Gpu Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

Tensorflow Version:2.20.0
Gpu Available: False


/Users/danherman/Desktop/headway-prediction/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# initialize configuration
config = Config()

# Fix data path: The notebook is in 'notebooks/', but data is in '../data'
config.DATA_DIR = os.path.join(os.path.abspath('..'), 'data')

# optional: Overrides settings for a quick test run
config.EPOCHS = 2
config.BATCH_SIZE = 32

print(f"Data directory: {config.DATA_DIR}")
print(f"Lookback: {config.LOOKBACK_MINS} min")
print(f"Forecast: {config.FORECAST_MINS} min")

Data directory: /Users/danherman/Desktop/headway-prediction/data
Lookback: 30 min
Forecast: 15 min


In [3]:
# initialize generator
data_gen = SubwayDataGenerator(config)

# load data from disk
data_gen.load_data()

# Calculate split indices (70% Train, 20% Val, 10% Test)
total_samples = len(data_gen.headway_data)
train_end = int(total_samples * 0.6)
val_end = int(total_samples * 0.8)

print(f"Total Samples: {total_samples}")
print(f"Train: 0 - {train_end}")
print(f"Val:   {train_end} - {val_end}")
print(f"Test:  {val_end} - {total_samples}")

# Create Datasets
train_ds = data_gen.make_dataset(start_index=0, end_index=train_end)
val_ds = data_gen.make_dataset(start_index=train_end, end_index=val_end)
test_ds = data_gen.make_dataset(start_index=val_end, end_index=total_samples)

# Verify shapes by pulling one batch
for inputs, targets in train_ds.take(1):
    print("\n--- Batch Shapes ---")
    print(f"Headway Input: {inputs[0].shape}")   # (Batch, 30, Stations, 2, 1)
    print(f"Schedule Input: {inputs[1].shape}")  # (Batch, 15, 2, 1)
    print(f"Target Output: {targets.shape}")     # (Batch, 15, Stations, 2, 1)

Loading data from /Users/danherman/Desktop/headway-prediction/data...
Headway Shape: (264222, 66, 2, 1)
Schedule Shape: (264222, 2, 1)
Total Samples: 264222
Train: 0 - 158533
Val:   158533 - 211377
Test:  211377 - 264222

--- Batch Shapes ---
Headway Input: (32, 30, 66, 2, 1)
Schedule Input: (32, 15, 2, 1)
Target Output: (32, 15, 66, 2, 1)


2026-01-05 12:03:37.020625: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# Initialize Model
print("Building Model...")
st_net = HeadwayConvLSTM(config)
model = st_net.build_model()

model.summary()

Building Model...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ headway_input       │ (None, 30, 66, 2, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (None, 30, 66, 2, │     38,144 │ headway_input[0]… │
│ (ConvLSTM2D)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 30, 66, 2, │        128 │ conv_lstm2d[0][0] │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_1       │ (None, 66, 2, 64) │    221,440 │ batch_normalizat… │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_state        │ (None, 15, 66, 2, │          0 │ conv_lstm2d_1[0]… │
│ (Lambda)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ schedule_input      │ (None, 15, 2, 1)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ broadcast_schedule  │ (None, 15, 66, 2, │          0 │ schedule_input[0… │
│ (Lambda)            │ 1)                │            │ repeat_state[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 15, 66, 2, │          0 │ repeat_state[0][… │
│ (Concatenate)       │ 65)               │            │ broadcast_schedu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_2       │ (None, 15, 66, 2, │    111,872 │ concatenate[0][0] │
│ (ConvLSTM2D)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ headway_output      │ (None, 15, 66, 2, │         33 │ conv_lstm2d_2[0]… │
│ (Conv3D)            │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 371,617 (1.42 MB)

 Trainable params: 371,553 (1.42 MB)

 Non-trainable params: 64 (256.00 B)

In [5]:
# Initialize Trainer
trainer = Trainer(model, config)
trainer.compile_model()

# Train
# We use the train/val datasets we created earlier
history = trainer.fit(train_ds, val_ds)

Starting training for 2 epochs...
Epoch 1/2
 140/4954 ━━━━━━━━━━━━━━━━━━━━ 1:07:24 840ms/step - loss: 0.0518 - mae: 0.1748

KeyboardInterrupt: 

In [ ]:
# Evaluate Training
evaluator = Evaluator(config)
evaluator.plot_loss(history)

In [ ]:
# Visualize Predictions on Test Set
print("Visualizing predictions on Test Set...")
evaluator.plot_spatiotemporal_prediction(model, test_ds, sample_idx=0)